# Proyecto: Clustering de Incidentes para Problem Management

Este cuaderno (notebook) construye un flujo completo de *Problem Management* aplicando **clustering** a un historial de tickets para descubrir **patrones ocultos** y generar **recomendaciones accionables**.

> **Nota:** Los datos son sintéticos, generados para fines educativos y de portafolio.


## Objetivos
- Identificar grupos de incidentes con características similares.
- Detectar comportamientos recurrentes (por categoría, prioridad, grupos asignados, departamento).
- Proponer acciones preventivas basadas en evidencia.


## Plan de trabajo
1. Cargar y explorar datos.
2. Ingeniería de características (tiempo de resolución, variables temporales).
3. Preparación de datos (codificación y escalado).
4. **K-Means**: selección de K (codo y *silhouette*), entrenamiento y visualización (PCA).
5. **DBSCAN**: estimación de `eps`, entrenamiento y visualización.
6. Perfilado de clústeres e **insights**.
7. Recomendaciones y *next steps*.


In [ ]:
# %% [markdown]
# # 1) Carga de librerías
# Configuración general en español
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['font.size'] = 11


In [ ]:
# %% [markdown]
# # 2) Cargar datos
# Si colocas el CSV en otra ruta, ajusta `csv_path`.
csv_path = 'incidentes_itsm.csv'
df = pd.read_csv(csv_path, parse_dates=['created_at', 'resolved_at'])
print('Registros:', len(df))
df.head()


In [ ]:
# %% [markdown]
# # 3) Exploración de datos (EDA)
print('Rango de fechas:', df['created_at'].min(), '->', df['created_at'].max())
print('
Distribución por prioridad:')
print(df['priority'].value_counts(dropna=False))
print('
Top categorías:')
print(df['category'].value_counts().head(10))
# Visualizaciones rápidas
fig, axes = plt.subplots(1, 2, figsize=(14,5))
sns.countplot(data=df, x='priority', order=sorted(df['priority'].unique()), ax=axes[0])
axes[0].set_title('Distribución de prioridades')
axes[0].set_xlabel('Prioridad')
axes[0].set_ylabel('Conteo')
top_cat = df['category'].value_counts().index
sns.countplot(data=df, y='category', order=top_cat, ax=axes[1])
axes[1].set_title('Incidentes por categoría')
axes[1].set_xlabel('Conteo')
axes[1].set_ylabel('Categoría')
plt.tight_layout()
plt.show()


In [ ]:
# %% [markdown]
# # 4) Ingeniería de características
# Tiempo de resolución en horas
df['resolution_time_hours'] = (df['resolved_at'] - df['created_at']).dt.total_seconds() / 3600.0
# Variables temporales
df['hora'] = df['created_at'].dt.hour
df['dia_semana'] = df['created_at'].dt.dayofweek  # 0=lunes, 6=domingo
df['es_fin_de_semana'] = df['dia_semana'].isin([5,6]).astype(int)
# Mapeo de prioridad a ordinal
priority_map = {'P1':1, 'P2':2, 'P3':3, 'P4':4}
df['priority_ord'] = df['priority'].map(priority_map)
# One-hot encoding para variables categóricas
cat_cols = ['category', 'sub_category', 'assigned_group', 'user_department']
df_enc = pd.get_dummies(df, columns=cat_cols, drop_first=True)
# Variables finales para clustering
features = ['priority_ord', 'resolution_time_hours', 'hora', 'dia_semana', 'es_fin_de_semana'] +     	[x for x in df_enc.columns if x.startswith('category_') or x.startswith('sub_category_') or x.startswith('assigned_group_') or x.startswith('user_department_')]
X = df_enc[features].copy()
# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3], X.shape


In [ ]:
# %% [markdown]
# # 5) K-Means: selección de K y entrenamiento
inertias = []
silhouettes = []
Ks = list(range(2, 11))
for k in Ks:
    km = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    sil = silhouette_score(X_scaled, labels)
    silhouettes.append(sil)
fig, ax1 = plt.subplots()
ax1.plot(Ks, inertias, marker='o')
ax1.set_xlabel('K')
ax1.set_ylabel('Inercia (distancia intra-clúster)')
ax1.set_title('Método del codo (K-Means)')
plt.show()
plt.figure()
plt.plot(Ks, silhouettes, marker='s', color='orange')
plt.xlabel('K')
plt.ylabel('Coeficiente de silhouette')
plt.title('Silhouette por número de clústeres')
plt.show()
# Selección automática por mejor silhouette
best_k = Ks[int(np.argmax(silhouettes))]
print('Mejor K por silhouette:', best_k)
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=50)
df['cluster_kmeans'] = kmeans.fit_predict(X_scaled)


In [ ]:
# %% [markdown]
# # 6) Visualización 2D (PCA)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)
df_plot = pd.DataFrame({
    'PC1': X_pca[:,0],
    'PC2': X_pca[:,1],
    'cluster_kmeans': df['cluster_kmeans']
})
sns.scatterplot(data=df_plot, x='PC1', y='PC2', hue='cluster_kmeans', palette='tab10')
plt.title('Clústeres K-Means proyectados con PCA')
plt.legend(title='Clúster')
plt.show()


In [ ]:
# %% [markdown]
# # 7) Perfilado de clústeres (K-Means)
# Resumen numérico
profile_num = df.groupby('cluster_kmeans')[['resolution_time_hours','priority_ord','hora','dia_semana']].agg(['mean','median','count']).round(2)
profile_num


In [ ]:
# Perfil categórico: top categoría, subcategoría y departamento por clúster
def top_n(series, n=3):
    vc = series.value_counts().head(n)
    return ', '.join([f"{idx} ({cnt})" for idx, cnt in vc.items()])
profile_cat = df.groupby('cluster_kmeans').agg({
    'category': top_n,
    'sub_category': top_n,
    'user_department': top_n,
    'assigned_group': top_n
})
profile_cat


In [ ]:
# %% [markdown]
# # 8) DBSCAN: estimación de eps y entrenamiento
# Usamos k-dist plot con k = min_samples - 1
min_samples = 10
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_scaled)
distances, _ = nn.kneighbors(X_scaled)
k_dist = np.sort(distances[:, -1])
plt.plot(k_dist)
plt.title('Gráfico k-dist (para estimar eps)')
plt.xlabel('Puntos ordenados')
plt.ylabel('Distancia al vecino k')
plt.show()
# Heurística: eps en percentil 90 de k_dist
eps = float(np.percentile(k_dist, 90))
print('eps (heurístico):', round(eps, 3))
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels_db = db.fit_predict(X_scaled)
df['cluster_dbscan'] = labels_db
print('Clases DBSCAN:', np.unique(labels_db))


In [ ]:
# Visualización DBSCAN con PCA
df_plot_db = pd.DataFrame({
    'PC1': X_pca[:,0],
    'PC2': X_pca[:,1],
    'cluster_dbscan': df['cluster_dbscan']
})
sns.scatterplot(data=df_plot_db, x='PC1', y='PC2', hue='cluster_dbscan', palette='tab20')
plt.title('DBSCAN proyectado con PCA (ruido = -1)')
plt.legend(title='Clúster')
plt.show()


In [ ]:
# %% [markdown]
# # 9) Insights automáticos y recomendaciones
insights = []
for c_id, g in df.groupby('cluster_kmeans'):
    top_cat = g['category'].value_counts().idxmax()
    med_time = g['resolution_time_hours'].median()
    top_dept = g['user_department'].value_counts().idxmax()
    top_group = g['assigned_group'].value_counts().idxmax()
    insights.append({
        'cluster': int(c_id),
        'resumen': f"Clúster {c_id}: predominan casos de {top_cat}, mediana de resolución {med_time:.1f} h, departamento más afectado {top_dept}, grupo asignado típico {top_group}.",
        'recomendacion': f"Revisar causas raíz en {top_cat} para {top_dept}. Coordinar con {top_group} mejoras específicas (runbooks, automatización, capacitación)."
    })
insights_df = pd.DataFrame(insights).sort_values('cluster')
insights_df


In [ ]:
# %% [markdown]
# # 10) Guardar resultados
out_clusters = 'incidentes_clusterizados_kmeans.csv'
out_profile = 'perfil_clusters_kmeans.csv'
df.to_csv(out_clusters, index=False, encoding='utf-8')
profile_combined = profile_num.copy()
profile_combined.columns = ['_'.join(col).strip() for col in profile_combined.columns.values]
profile_final = profile_combined.merge(profile_cat, left_index=True, right_index=True)
profile_final.to_csv(out_profile, encoding='utf-8')
print('Archivos guardados:', out_clusters, 'y', out_profile)


## Siguientes pasos
- Validar con expertos si los clústeres tienen sentido operativo.
- Profundizar en causa raíz por clúster (5 porqués, Ishikawa).
- Crear **runbooks** y **automatizaciones** para clústeres de alto volumen.
- Conectar con datos reales del Service Desk (Jira/ServiceNow) y programar ejecución semanal.
